
# Project: FBI Gun Data Analysis

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

> For this project, I have chosen the data from the FBI's National Instant Criminal Background Check System (NICS) and the U.S Census Data. The NICS is used to determine whether a prospective buyer is eligible to buy firearms or explosives. The statistics in this dataset originally represent the number of firearm background checks initiated through the NICS. They do not represent the number of firearms sold. HOWEVER, ACCORDING TO "The Trace (2015), THE FBI'S NICS NUMBERS ARE WIDELY ACCEPTED AS THE BEST PROXY FOR TOTAL GUN SALES IN A GIVEN TIME PERIOD. All analysis that will be carried out with this dataset will be considered related to Gun Sales.

>The U.S Census Data is also to provide demographic data about the U.S States. Statistics from this dataset will be referenced in this project for more insight.

>I aim to answer four (4) questions as I Investigate this dataset.
>1. What is the trend of Gun Sales across the U.S States?
>2. Can the Per Capita Income of a State reveal how guns are Purchased?
>3. Are the number of veterans in a State indicative of guns in the state?
>4. What is the Race distribution of the State with the highest number of Guns

In [ ]:
# Importing all packages to be used in the project.

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

<a id='wrangling'></a>
## Data Wrangling

> In this section of the project, the datasets will be loaded. The structure will be noted and inspected for the need for any cleaning.

### General Properties

> **FBI Gun Data will be read into a dataframe(df_gun) and U.S Census Data will be read into another dataframe(df_census). The first 5 data entries in each dataframe is displayed to have an idea of the structure.**

In [ ]:
df_gun = pd.read_excel('gun_data.xlsx')
df_census = pd.read_csv('U.S. Census Data.csv')

In [ ]:
df_gun.head()

In [ ]:
df_census.head()

>**The total number of rows and columns is revealed for both dataframes with the .shape() method and the .info() is also used to inspect for the datatypes, the columns and any missing values. By doing these, I start to identify which rows and columns will be necessary.**

In [ ]:
# Finding out the total rows and columns in the gun_data dataset
df_gun.shape

In [ ]:
# Finding out the total rows and columns in the U.S Census Data dataset
df_census.shape

In [ ]:
# Dataset information for gun_data
df_gun.info()

In [ ]:
# Dataset information for census data
df_census.info()

>**df_gun has 12485 rows made up of each month from September 2017 to November 1998. The 27 columns have data on the kind of guns or permits called in for check.**
>**df_census has 85 rows of varied demographic into and 52 columns of the States. Most of the datatypes are string which is quite strange for a census data. This will be explore more.**

> As mentioned in the introduction, the statistics in the FBI's NICS represent the number of firearm background checks intiated and not necessarily the firearms sold BUT these numbers have been widely accepted as the best proxy for total gun sales in a given period.
> When the New York Times analyzed NICS data in Dec. 2015, it included this methodology note:
> Sales estimates are calculated from handgun, long gun and multiple-gun background checks. Permit checks and other categories of background checks are excluded.

> For this project, the Sales estimates will be calculated from handgun, long gun and other since these three (3) columns give of all types of firearm. The multiple-gun will be excluded since it is just indicative of checks involving more than one fire arm. The other categories(columns) will also be excluded.

### Data Cleaning (FBI Gun Data - df_gun)

In [ ]:
# Creating a list of columns which will not be needed and dropping them.

non_essential_columns = ['permit', 'permit_recheck','admin','prepawn_handgun','prepawn_long_gun',
                        'prepawn_other','redemption_handgun','redemption_long_gun','redemption_other',
                        'returned_handgun','returned_long_gun','returned_other','rentals_handgun',
                        'rentals_long_gun','private_sale_handgun','private_sale_long_gun',
                        'private_sale_other','return_to_seller_handgun','return_to_seller_long_gun',
                         'return_to_seller_other']
df_gun.drop(non_essential_columns, axis=1, inplace=True)

In [ ]:
# Confirm the columns in the list above have been dropped.
df_gun.head()

In [ ]:
# Since most of the data in the U.S Census Data is referenced to the year 2016,
# This dataset will also be limited to the year 2016 for more accurate analysis
# All rows of data for years other than 2016 will be dropped

df_gun.drop(df_gun.index[df_gun['month'].str.contains("2016")==False], inplace=True)

In [ ]:
#Confirm the rows have been dropped.
df_gun.head()

In [ ]:
# To have one row of data for each State, the dataset is grouped by state and all values for each
# state is summed up 

df_gun = df_gun.groupby(['state'], as_index=False).sum()

In [ ]:
# Confirming the sum of values have been made according to the states
df_gun.head()

In [ ]:
# Checking number of rows to give an idea of the number of states.
df_gun.shape

In [ ]:
# The rows are 55, which is more than the known 50 Federal Republic States.
# Checking for any duplicated States
df_gun.duplicated().sum()

>**The df_gun has been reduced to 6 columns (state, handgun, long_gun, other, multiple, totals). No data entry has been changed yet. The expected number of rows was 51 to correspond with the 50 Federeal States of the USA but it has turned out to be more. Probably some territories have been added. This will be dropped later after cleaning df_census.**

> As Seen in the display of general properties, the US Census Data has information about each state as well with about 80 different fields. To answer the questions set at the beginning of the project, some rows will be dropped. Only rows relevant to the projects research questions will be kept.

>Also, in this dataset, the states appear as the columns rather. This will be transposed to fit the same structure as the df_gun(states are the rows) for easier merging of the two datasets

>The new column names will be changed as they are too long and will make subsequent codes more difficult. Each column name will be changed to lowercase and underscores used to seperate multiple words.

### Data Cleaning (US Census Data - df_census)

In [ ]:
# Rows that will be essential in answering the research questions = [0, 12, 13, 14, 15, 16, 20, 48]
# Extracting these rows out of the lot
df_census = df_census.loc[[0, 12, 13, 14, 15, 16, 20, 48],:]

In [ ]:
# Confirming the essential rows have been extracted
df_census

In [ ]:
# This column was added to provided extra information on the rows
# Dropping the 'Fact Note' column
df_census.drop('Fact Note', axis=1, level=None, inplace=True)

In [ ]:
# Setting Fact Column as Index
df_census = df_census.set_index('Fact')

In [ ]:
# Transposing the dataset to have the same the states as rows rather than columns
# Similar to df_gun dataset.
df_census = df_census.transpose()

In [ ]:
# Reseting indexes
df_census.reset_index(inplace=True)

In [ ]:
# Confirming the drop,transpose and index reset
df_census.head()

In [ ]:
# Assigning the state label to have the same label in df_gun
df_census = df_census.rename(columns = {'index':'state'})

In [ ]:
df_census.head()

>**The essential rows in df_census have been extracted, 'Fact Note' Column dropped and the dataset transposed so we have the same structure with df_gun. The states are now common with df_gun.**

> Now that both df_gun and df_census are similar in structure, they will be merged to form a new dataframe df_merged

>The column names of df_merged will be simplified to lower_case and the datatypes changed to help in analysis. As mentioned earlier, most of the datatypes from the US Census Data were string. They are better as int and floats for computations sake.

>One Column has "Z" in the entries, according to the Fact Notes of the original US Census dataset, this Z signified value greater than zero but less than half unit of measure. The other entries in that column are very small and so this "Z" in reality is almost 0. 

### Data Cleaning (New Dataframe Consisting of both df_gun & df_census)

In [ ]:
# Merging of df_gun and df_census on the state column.
# Inner Join is used so df_merged will have only states common to both df_gun & df_census

df_merged=df_gun.merge(df_census, left_on='state', right_on='state', how='inner')

In [ ]:
# Renaming of column headers to make them easy to call 
df_merged.rename(columns = {'Population estimates, July 1, 2016,  (V2016)':'population_estimates',
                 'White alone, percent, July 1, 2016,  (V2016)':'white',
                'Black or African American alone, percent, July 1, 2016,  (V2016)':'black_or_african_american',
                'American Indian and Alaska Native alone, percent, July 1, 2016,  (V2016)':'native_american',
                'Asian alone, percent, July 1, 2016,  (V2016)':'asian',
                 'Native Hawaiian and Other Pacific Islander alone, percent, July 1, 2016,  (V2016)':
                 'hawaiian_or_islanders','Veterans, 2011-2015':'veterans_11_15',
                 'Per capita income in past 12 months (in 2015 dollars), 2011-2015':'per_capita_income_11_15'},
                inplace=True)
df_merged.head(0)

In [ ]:
# Confirming the merging and renaming of columns has worked.
df_merged.head()

In [ ]:
#Checking the properties of df_merged
df_merged.info()

In [ ]:
# The following columns (handgun,long_gun and other) represent a count of the various types of gun
# and so the datatype will be changed to int
# Changing the datatype to int
df_merged = df_merged.astype({'handgun':'int64','long_gun':'int64','other':'int64'})

In [ ]:
# The following are stored as object but need to be changed to float as they represent percentages
# Changing the datatypes to float.

df_merged['white'] = df_merged['white'].str.rstrip("%").astype(float).round(2);
df_merged['black_or_african_american'] = df_merged['black_or_african_american'].str.rstrip("%").astype(float).round(2);
df_merged['native_american'] = df_merged['native_american'].str.rstrip("%").astype(float).round(2);
df_merged['asian'] = df_merged['asian'].str.rstrip("%").astype(float).round(2);

In [ ]:
# The following are stored as object but need to be changed to int as dey represent counts
# Changing to datatypes to int

df_merged['veterans_11_15'] = df_merged['veterans_11_15'].str.replace(',','').astype(int);
df_merged['population_estimates'] = df_merged['population_estimates'].str.replace(',','').astype(int);

In [ ]:
# The per_capita_income_11_15 is a currency value and needs to be changed to float for computations.
# Stripping the extra characters and changing datatype to float

df_merged['per_capita_income_11_15'] = df_merged['per_capita_income_11_15'].str.strip();
df_merged['per_capita_income_11_15'] = df_merged['per_capita_income_11_15'].str.strip("$");
df_merged['per_capita_income_11_15'] = df_merged['per_capita_income_11_15'].str.replace(',','').astype(float);

In [ ]:
# Identifying the index of all data entries with "Z"
df_merged.index[df_merged['hawaiian_or_islanders'] =='Z'].tolist()

In [ ]:
# Replacing all 'Z' with 0 as explained in the previous Markdown.
# Changing the datatype to float

df_merged.at[18,'hawaiian_or_islanders']=0
df_merged.at[21,'hawaiian_or_islanders']=0
df_merged.at[44,'hawaiian_or_islanders']=0
df_merged.at[47,'hawaiian_or_islanders']=0
df_merged['hawaiian_or_islanders'] = df_merged['hawaiian_or_islanders'].str.strip("%").astype(float);

In [ ]:
# Ooops, Almost forgot the totals column has to be computed right.
# The totals column will be the sum of handgun, long_gun and others. This is the total firearm

df_merged['totals'] = df_merged.iloc[:,1:3].sum(axis=1)

>**The new dataframe df_merged has been cleaned and is read to be analysed and the research questions answered.**

<a id='eda'></a>
## Exploratory Data Analysis

>Here we go with the visualizations and answers to the research questions !!

### Research Question 1
### (What is the Trend of Gun Sales across the U.S States?)

In [ ]:
#handgun,long_gun & others
#ax=df_merged.plot(x='state', y='handgun', kind='bar', title='Number of Checks by State', figsize=(16,10))
#df_merged.plot(x='state', y='long_gun', kind='bar', ax=ax, title='Number of Checks by State', figsize=(16,10), color='red')
#df_merged.plot(x='state', y='other', kind='bar', ax=ax, title='Number of Checks by State', figsize=(16,10), color='green')

#plt.xlabel('States')
#plt.ylabel('Number of Checks')

>**This question requires we see each states gun sales.**

>**We plot a bar graph of 'totals' against the 'state'.**

In [ ]:
#Plotting a bar graph of US States and Total Gun Sales
df_merged.plot(x='state', y='totals', kind='bar', title='Gun Sales', figsize=(16,10), color='brown')

>Interesting things to note about this plot.

>1. Hawaii has no Gun Sales. (No calls to check the NICS)

>2. Texas and California States competing for the highest Gun Sales.

> The above plot shows the Total Gun Sales of the US States. However, the States are not of the same size in terms of population. The next plot is just to show the trend of Gun Sales with respect to the number of people in the State. 

In [ ]:
# This computation is to show the Gun Sales with respect to the Population of each state
# It represents the proportion of the population who have contributed to the gun sales

df_merged['gun_per_population'] = df_merged['totals']/df_merged['population_estimates']
df_merged.plot(x='state', y='gun_per_population', kind='bar', title='Guns Per Population', figsize=(16,10), color='blue')

> The graph changes once we consider the population. 10% of people in Alaska have contributed to the Gun Sales whereas 4% of people in Texas have done same.

### Research Question 2  
### (Can the Per Capita Income of a State reveal how Guns are Purchased ?)

>**Since guns arent cheap, it is easy to assume a certain economic state will have to be attained before they can be bought. This question requires we find a way to plot the per capita income and total gun sales to be able to make an inference**

In [ ]:
# Using seaborn, we plot a scatterplot and differentiate the markers using the total sale so that
# the states with bigger total sales will actually look bigger and darker in colour.
# Then we can tell the per_capita_income of these states

plt.figure(figsize=(16,10))
plt.grid(True)
plt.ylim()
sns.scatterplot(data=df_merged, x='per_capita_income_11_15', y='totals',
                size='totals', sizes=(20,1000), hue='totals')

>As can be infered from the scatter plot, higher per capita income doesn't necessarily mean more gun sales. The states with the most gun sales(bigger markers) ended up in the median per capital income range. 

### Research Question 3  
### (Are the veterans living in a state indicative of the guns purchases?)

>**For this question, it is best to plot the state against the total guns on the same bar as number of veterans.

In [ ]:
ax=df_merged.plot(x='state', y='veterans_11_15', kind='bar', title='Gun Sales  & Number of Veterans by State',
               figsize=(16,10), color ='yellow', alpha=0.5)
df_merged.plot(x='state', y='totals', kind='bar', title='Number of Checks by State', ax=ax, figsize=(16,10),alpha=0.7)
plt.grid(True)

>It cannot be stated emphatically that the veterans influence the gun sales in the states. This is because the bars are not largely similar among all states. Some states have more veterans than gun sale and more states have more veterans than gun sales. 

### Research Question 4  
### (What is the Race distribution in the States with highest Gun Purchases ?)

In [ ]:
#Find state with the highest gun sale by sorting the values
top_gun_states=df_merged.sort_values(by=['totals'])

In [ ]:
#Confirming the name of the state
top_gun_states

In [ ]:
#Texas
texas_values=top_gun_states.iloc[42,7:12]
texas_labels=top_gun_states.columns[7:12]
plt.pie(texas_values, labels=texas_labels,radius=2)
plt.show()

>The pie chart reveals a large percentage of the population of Texas is white followed by black_or_african_american and then asian.

<a id='conclusions'></a>
## Conclusions

> The FBI Gun Data has bee analysed to answer four questions. This was done in conjunction with the US Census Data. At the end of the project, we have been able to find the trend in gun sales in the US and also gained insight on per capita income and the gun sale distribution. 

> No statistical inferences have been made to explain the answers to the questions. 

> One limitation of this project is the fact that a huge chunk of the data had to be dropped to be able to continue with the analysis successfully. A considerable part of the data was non-essential.
